# Plasmodium falciparum Drug Resistance in Nigeria – Figure & Analysis Notebook

**Systematic Review & Meta-analysis (2000–2024, extension-ready for 2025)**  
Author: *Khadim Gueye*  
Last updated: 2025

---
**What this notebook does**
- Loads the curated supplementary tables (S1–S6).
- Generates all main and supplementary figures for the manuscript:
  - Figure 1 – PRISMA flow diagram (semi-automated, publication-ready style)
  - Figure 2A–D – Study metrics
  - Figure 3 – pfcrt K76T over time
  - Figure 4A–B – pfmdr1 mutations
  - Figure 5A–D – pfdhfr & pfdhps mutations and haplotypes
  - Figure S1 – additional pfdhps mutations
  - Figure 6 – kelch13 mutations


## 1. Setup – Imports & Global Plot Style

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path


# Optional: advanced stats & mapping packages (install if missing)
try:
    import statsmodels.api as sm
    from scipy import stats
except ImportError:
    print('statsmodels / scipy not installed – install if you want regression & χ² tests.')

try:
    import geopandas as gpd
except ImportError:
    print('geopandas not installed – install if you want state-level maps (Figure 7).')

warnings.filterwarnings('ignore')

sns.set(style='whitegrid', context='talk')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

print('Libraries imported and plotting style set.')

## 2. Load Supplementary Tables (S1–S6)

Adjust paths if your files live somewhere else. This notebook assumes:
- `Table_S1_included_studies.xlsx`
- `Table_S2_pfcrt.xlsx`
- `Table_S3_pfmdr1.xlsx`
- `Table_S4_pfdhfr_pfdhps.xlsx`
- `Table_S5_pfkelch13.xlsx`
- `Table_S6_pfcoronin.xlsx`

Run this cell once you have these files in place.

In [ ]:
df_studies = pd.read_excel('new/Table_S1_included_studies.xlsx')
df_pfcrt = pd.read_excel('new/Table_S2_pfcrt.xlsx', header=1)
df_pfmdr1 = pd.read_excel('new/Table_S3_pfmdr1.xlsx')
S4_PATH = Path("data/Table_S4_pfdhfr_pfdhps.xlsx")
S5_PATH = Path("data/Table_S5_pfkelch13.xlsx")
df_pfcoronin = pd.read_excel('new/Table_S6_pfcoronin.xlsx')


## 3. Figure 1 – Automated PRISMA Flow Diagram

Because PRISMA numbers depend on your exact screening process, we define them as editable variables.

Fill in the numbers below based on your final study selection log, then run the plotting cell to generate a
vector-style PRISMA diagram using Matplotlib shapes (suitable for export as SVG or high-res PNG).

In [ ]:
# EDIT THESE VALUES ACCORDING TO YOUR REVIEW
prisma_counts = {
    'records_identified': 0,      # e.g. from database search
    'records_duplicate_removed': 0,
    'records_screened_title_abstract': 0,
    'records_excluded_title_abstract': 0,
    'fulltext_assessed': 0,
    'fulltext_excluded': 0,
    'studies_included_qualitative': df_studies.shape[0],  # usually 80
    'studies_included_quantitative': df_studies.shape[0]  # if same as qualitative
}

prisma_counts

In [ ]:
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

def draw_box(ax, xy, text, boxstyle='round,pad=0.3', box_kwargs=None):
    if box_kwargs is None:
        box_kwargs = dict(boxstyle=boxstyle, facecolor='white', edgecolor='black')
    ax.text(xy[0], xy[1], text, ha='center', va='center', bbox=box_kwargs, fontsize=11)

def draw_arrow(ax, xy_from, xy_to):
    arrow = FancyArrowPatch(xy_from, xy_to, arrowstyle='-|>', mutation_scale=15,
                            linewidth=1.5, color='black')
    ax.add_patch(arrow)

fig, ax = plt.subplots(figsize=(8, 10))
ax.set_axis_off()

y_top = 0.9
y_step = 0.15

t0 = f"Records identified through database searching (n = {prisma_counts['records_identified']})"
t1 = f"Records after duplicates removed (n = {prisma_counts['records_duplicate_removed']})"
t2 = f"Records screened (title/abstract) (n = {prisma_counts['records_screened_title_abstract']})"
t3 = f"Records excluded (n = {prisma_counts['records_excluded_title_abstract']})"
t4 = f"Full-text articles assessed for eligibility (n = {prisma_counts['fulltext_assessed']})"
t5 = f"Full-text articles excluded (n = {prisma_counts['fulltext_excluded']})\nwith reasons"
t6 = ("Studies included in qualitative synthesis\n"
      f"(n = {prisma_counts['studies_included_qualitative']})")
t7 = ("Studies included in quantitative synthesis\n"
      f"(meta-analysis, n = {prisma_counts['studies_included_quantitative']})")

x_center = 0.5

positions = {
    't0': (x_center, y_top),
    't1': (x_center, y_top - y_step),
    't2': (x_center, y_top - 2*y_step),
    't3': (x_center + 0.3, y_top - 2*y_step),
    't4': (x_center, y_top - 3*y_step),
    't5': (x_center + 0.3, y_top - 3*y_step),
    't6': (x_center, y_top - 4*y_step),
    't7': (x_center, y_top - 5*y_step)
}

draw_box(ax, positions['t0'], t0)
draw_box(ax, positions['t1'], t1)
draw_box(ax, positions['t2'], t2)
draw_box(ax, positions['t3'], t3)
draw_box(ax, positions['t4'], t4)
draw_box(ax, positions['t5'], t5)
draw_box(ax, positions['t6'], t6)
draw_box(ax, positions['t7'], t7)

draw_arrow(ax, (x_center, positions['t0'][1]-0.03), (x_center, positions['t1'][1]+0.03))
draw_arrow(ax, (x_center, positions['t1'][1]-0.03), (x_center, positions['t2'][1]+0.03))
draw_arrow(ax, (x_center, positions['t2'][1]-0.03), (x_center, positions['t4'][1]+0.03))
draw_arrow(ax, (x_center, positions['t4'][1]-0.03), (x_center, positions['t6'][1]+0.03))
draw_arrow(ax, (x_center, positions['t6'][1]-0.03), (x_center, positions['t7'][1]+0.03))

draw_arrow(ax, (x_center+0.12, positions['t2'][1]), (positions['t3'][0]-0.12, positions['t3'][1]))
draw_arrow(ax, (x_center+0.12, positions['t4'][1]), (positions['t5'][0]-0.12, positions['t5'][1]))

plt.title('Figure 1. PRISMA Flow Diagram', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

## 4. Figure 2 – Publication Metrics (A–D)

In [ ]:
gene_cols = [c for c in ['pfcrt','pfmdr1','pfdhfr','pfdhps','pfkelch13','pfcoronin'] if c in df_studies['molecular marker']]
markers_series = (
    df_studies['molecular marker']
    .dropna()
    .str.split(',')
    .explode()
    .str.strip()
    .str.lower()
)
gene_counts = markers_series.value_counts().sort_values(ascending=False)
gene_counts

In [ ]:

# -----------------
# Style helpers
# -----------------
def style_axes(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.tick_params(axis='both', width=2, length=7)

def add_bar_labels(ax, dy=0.8):
    for p in ax.patches:
        h = p.get_height()
        if pd.notna(h) and h > 0:
            ax.text(p.get_x() + p.get_width()/2, h + dy, f"{int(h)}",
                    ha='center', va='bottom', fontsize=16, fontweight='bold')

# -----------------
# Study focus -> category (pie)
# -----------------
def classify_focus(x):
    if pd.isna(x):
        return 'NA or mixed'
    t = str(x).strip().lower()

    if 'returning traveler' in t:
        return 'returning travelers'
    if 'sca' in t:
        return 'SCA'
    if 'hiv' in t:
        return 'HIV +/-'
    if 'pregnan' in t:
        return 'pregnant women'
    if 'child' in t or '(children)' in t or '<5' in t or '< 5' in t or 'months' in t:
        return 'children'

    # mixed / all ages / adults / NA
    if t == 'na' or 'mixed' in t or 'all ages' in t or 'adults' in t:
        return 'NA or mixed'

    return 'NA or mixed'

# -----------------
# Figure layout
# -----------------
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
(axA, axB), (axC, axD) = axes

# =========================================================
# 2A – number of publications per molecular marker (colored)
# =========================================================
if 'molecular marker' in df_studies.columns:

    markers_series = (
        df_studies['molecular marker']
        .dropna()
        .astype(str)
        .str.split(',')
        .explode()
        .str.strip()
        .str.lower()
    )

    gene_counts = markers_series.value_counts()

    # order + colors like your sample
    marker_order = ['pfcoronin', 'pfdhps', 'pfkelch13', 'pfdhfr', 'pfmdr1', 'pfcrt']
    marker_order = [m for m in marker_order if m in gene_counts.index]

    marker_colors = {
        'pfcoronin': 'black',
        'pfdhps': 'blue',
        'pfkelch13': 'lime',
        'pfdhfr': 'magenta',
        'pfmdr1': 'gray',
        'pfcrt': 'white'
    }

    valsA = [int(gene_counts[m]) for m in marker_order]
    colsA = [marker_colors.get(m, 'white') for m in marker_order]

    axA.bar(marker_order, valsA, color=colsA, edgecolor='black', linewidth=2, width=0.65)

    axA.set_ylabel('number of publications', fontweight='bold' , fontsize=16)
    axA.set_xlabel('')
    axA.set_title('') 
    axA.tick_params(axis='x', rotation=55)
    axA.set_ylim(0, max(valsA) * 1.25)

    style_axes(axA)
    add_bar_labels(axA, dy=max(valsA)*0.02)
    axA.text(0.01, 0.98, "A", transform=axA.transAxes, fontsize=14, fontweight="bold", va="top")

else:
    axA.text(0.5, 0.5, 'molecular marker column not found', ha='center', va='center')
    axA.set_title('')

# ==================================
# 2B – study focus (PIE chart)
# ==================================
if 'study focus' in df_studies.columns:
    focus_cat = df_studies['study focus'].apply(classify_focus)
    focus_counts = focus_cat.value_counts()

    pie_order = ['returning travelers', 'SCA', 'pregnant women', 'children', 'HIV +/-', 'NA or mixed']
    pie_order = [k for k in pie_order if k in focus_counts.index]
    focus_counts = focus_counts.reindex(pie_order)

    pie_colors = {
        'returning travelers': 'black',
        'SCA': 'gray',
        'pregnant women': 'blue',
        'children': 'magenta',
        'HIV +/-': 'white',
        'NA or mixed': 'lime'
    }
    colsB = [pie_colors[k] for k in focus_counts.index]

    wedges, texts, autotexts = axB.pie(
        focus_counts.values,
        colors=colsB,
        startangle=90,
        counterclock=False,
        wedgeprops=dict(edgecolor='black', linewidth=1.5),
        autopct=lambda p: f'{p:.0f}%' if p >= 4 else ''  
    )

    axB.legend(wedges, focus_counts.index, loc='upper left',
               bbox_to_anchor=(1.02, 1.0), frameon=False)

    axB.text(0, -1.25, f"Total={int(focus_counts.sum())}",
             ha='center', va='top', fontsize=16, fontweight='bold')

    axB.set_title('')  # no title
    axB.axis('equal')
    axB.text(0.01, 0.98, "B", transform=axB.transAxes, fontsize=14, fontweight="bold", va="top")

else:
    axB.text(0.5, 0.5, 'study focus column not found', ha='center', va='center')
    axB.set_title('')

# ==================================
# 2C – country of sample collection (bars like sample)
# ==================================
if 'country' in df_studies.columns:
    country_counts = df_studies['country'].dropna().astype(str).value_counts()

    # keep top countries (sample figure shows few)
    country_counts = country_counts.head(10)

    def country_color(name):
        n = name.strip().lower()
        if 'nigeria' in n:
            return 'magenta'
        if 'china' in n:
            return 'lime'
        if 'france' in n:
            return 'blue'
        if 'italy' in n:
            return 'gray'
        return 'white'

    colsC = [country_color(c) for c in country_counts.index]

    axC.bar(country_counts.index, country_counts.values,
            color=colsC, edgecolor='black', linewidth=2, width=0.65)

    axC.set_ylabel('number of publications', fontweight='bold', fontsize=16)
    axC.set_xlabel('country of sample collection', fontweight='bold', fontsize=16)
    axC.set_title('')  # no title
    axC.tick_params(axis='x', rotation=40)
    axC.set_ylim(0, max(country_counts.values) * 1.25)

    style_axes(axC)
    add_bar_labels(axC, dy=max(country_counts.values)*0.02)
    axC.text(0.01, 0.98, "C", transform=axC.transAxes, fontsize=14, fontweight="bold", va="top")

else:
    axC.text(0.5, 0.5, 'country column not found', ha='center', va='center')
    axC.set_title('')

# ==================================
# 2D – number of studies by year (line)
# ==================================
if 'corrected year sample collection' in df_studies.columns:

    years_series = (
        df_studies['corrected year sample collection']
        .dropna()
        .astype(str)
        .str.split(',')
        .explode()
        .str.strip()
    )

    years_series = years_series[years_series.str.isnumeric()].astype(int)
    year_counts = years_series.value_counts().sort_index()

    axD.plot(year_counts.index, year_counts.values, marker='o',
             color='black', linewidth=1)

    axD.set_ylabel('number of studies', fontweight='bold', fontsize=16)
    axD.set_xlabel('year', fontweight='bold', fontsize=16)
    axD.set_title('')  # no title
    axD.set_ylim(0, max(year_counts.values) * 1.1)
    axD.set_xlim(2000)

    style_axes(axD)
    axD.text(0.01, 0.98, "D", transform=axD.transAxes, fontsize=14, fontweight="bold", va="top")

else:
    axD.text(0.5, 0.5, 'corrected year sample collection column not found',
             ha='center', va='center')
    axD.set_title('')

plt.tight_layout()
plt.show()
plt.savefig('plot/figure_2_study_characteristics.png', dpi=300)


## 5. Figure 3 – pfcrt K76T Prevalence Over Time

In [ ]:
# ---- Figure 3: pfcrt K76T prevalence over time (sample-plot style) ----

df_pfcrt_plot = df_pfcrt.copy()

# --- Normalize column names robustly (in case they differ a bit)
col_year = 'Year' if 'Year' in df_pfcrt_plot.columns else 'corrected year'
col_prev = 'Prevalence' if 'Prevalence' in df_pfcrt_plot.columns else 'prevalence K76T'
col_n    = 'Sample_Size' if 'Sample_Size' in df_pfcrt_plot.columns else 'sample size Nigeria only'

# --- Clean year
df_pfcrt_plot[col_year] = pd.to_numeric(df_pfcrt_plot[col_year], errors='coerce')
df_pfcrt_plot = df_pfcrt_plot.dropna(subset=[col_year])

# --- Clean prevalence: accept "43%", "0.43", "43", etc.
def to_float_prev(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = s.replace('%', '').strip()
    try:
        return float(s)
    except:
        return None

df_pfcrt_plot[col_prev] = df_pfcrt_plot[col_prev].apply(to_float_prev)
df_pfcrt_plot = df_pfcrt_plot.dropna(subset=[col_prev])

# if values look like percentages (0-100), convert to 0-1
if df_pfcrt_plot[col_prev].max() > 1.5:
    df_pfcrt_plot[col_prev] = df_pfcrt_plot[col_prev] / 100.0

# keep in [0,1]
df_pfcrt_plot[col_prev] = df_pfcrt_plot[col_prev].clip(0, 1)

# --- Optional: weights by sample size (if available & numeric)
if col_n in df_pfcrt_plot.columns:
    df_pfcrt_plot[col_n] = pd.to_numeric(df_pfcrt_plot[col_n], errors='coerce')

def weighted_mean(x):
    # weighted by sample size if available, else simple mean
    if col_n in x and x[col_n].notna().any():
        w = x[col_n].fillna(0)
        if w.sum() > 0:
            return (x[col_prev] * w).sum() / w.sum()
    return x[col_prev].mean()

def weighted_se(x):
    # simple SEM (study-level) for error bars; if only 1 study in year => NaN (no errorbar)
    return x[col_prev].sem()

df_pfcrt_year = (
    df_pfcrt_plot
    .groupby(col_year, as_index=True)
    .apply(lambda g: pd.Series({
        'mean_prevalence': weighted_mean(g),
        'se': weighted_se(g),
        'n_studies': g.shape[0]
    }))
    .sort_index()
)

# --- Plot
fig, ax = plt.subplots(figsize=(7.2, 4.2))

# error bars only where SE exists (>=2 studies in that year typically)
y = df_pfcrt_year['mean_prevalence'].values
x = df_pfcrt_year.index.values
yerr = df_pfcrt_year['se'].values

ax.errorbar(
    x, y,
    yerr=yerr,
    fmt='o-',
    color='black',
    ecolor='black',
    elinewidth=1.2,
    capsize=4,
    markersize=8,
    markerfacecolor='black',
    markeredgecolor='black',
    linewidth=1
)

# --- Horizontal dotted thresholds like sample plot
for yline in [1.0, 0.7, 0.2]:
    ax.axhline(yline, linestyle=':', color='black', linewidth=2)

ax.set_ylim(0, 1.02)

# Labels like sample
ax.set_xlabel('year', fontsize=16, fontweight='bold', labelpad=10)
ax.set_ylabel('prevalence of K76T mutation', fontsize=16, fontweight='bold', labelpad=10)

# Match tick feel
ax.tick_params(axis='both', which='major', width=3, length=10, labelsize=14)

# Thick spines like sample
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(3.5)
ax.spines['bottom'].set_linewidth(3.5)

# Set y ticks like sample (0, .25, .5, .75, 1.0)
ax.set_yticks([0, 0.25, 0.50, 0.75, 1.00])
ax.set_yticklabels([f"{t:.2f}" for t in [0, 0.25, 0.50, 0.75, 1.00]])

ax2 = ax.twinx()
ax2.set_ylim(ax.get_ylim())
ax2.set_yticks([1.0, 0.7, 0.2])
ax2.set_yticklabels(['1.0', '0.7', '0.2'])
ax2.tick_params(axis='y', which='major', width=0, length=0, labelsize=16)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

ax.set_title('')
ax.grid(False)

plt.tight_layout()
plt.show()
plt.savefig("plot/figure_3_pfcrt_K76T_prevalence_over_time.png", dpi=300)


## 6. Figure 4 – pfmdr1 Mutation Prevalence (A–B)

In [ ]:
# ---- Figure: pfmdr1 (sample-plot style) ----

df_pfmdr_plot = df_pfmdr1.copy()

# --- column confirmation (robust)
col_year = 'Year'
if col_year not in df_pfmdr_plot.columns:
    for cand in ['corrected year of sample collection', 'corrected year', 'corrected year sample collection']:
        if cand in df_pfmdr_plot.columns:
            col_year = cand
            break

mut_cols = ['N86Y', 'Y184F', 'S1034C', 'N1042D', 'D1246Y']
mut_cols = [c for c in mut_cols if c in df_pfmdr_plot.columns]

# --- clean year
df_pfmdr_plot[col_year] = pd.to_numeric(df_pfmdr_plot[col_year], errors='coerce')
df_pfmdr_plot = df_pfmdr_plot.dropna(subset=[col_year])

# --- clean prevalence cells: accept "70%", "0.70", "0.76%" etc.
def to_float_prev(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = s.replace('%', '').strip()
    try:
        return float(s)
    except:
        return None

for c in mut_cols:
    df_pfmdr_plot[c] = df_pfmdr_plot[c].apply(to_float_prev)

# if looks like percentage, convert to 0-1
mx = df_pfmdr_plot[mut_cols].max().max() if mut_cols else 0
if pd.notna(mx) and mx > 1.5:
    df_pfmdr_plot[mut_cols] = df_pfmdr_plot[mut_cols] / 100.0

df_pfmdr_plot[mut_cols] = df_pfmdr_plot[mut_cols].clip(0, 1)

# --- styling helpers
def style_axes(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(3)
    ax.spines['bottom'].set_linewidth(3)
    ax.tick_params(axis='both', which='major', width=3, length=10, labelsize=12)

def add_labels_on_bars(ax, dy=0.03):
    for p in ax.patches:
        h = p.get_height()
        if pd.notna(h):
            ax.text(p.get_x() + p.get_width()/2, h + dy, f"{h:.2f}",
                    ha='center', va='bottom', fontsize=11)

# --- colors (match sample)
bar_colors = {
    'N86Y': 'black',
    'Y184F': 'blue',
    'S1034C': 'white',
    'N1042D': 'lime',
    'D1246Y': 'gray'
}

# --- figure
fig, (axA, axB) = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'wspace': 0.35})

# ======================
# Left panel: mean + SE barplot
# ======================
if mut_cols:
    mean_vals = df_pfmdr_plot[mut_cols].mean(skipna=True)
    se_vals = df_pfmdr_plot[mut_cols].sem(skipna=True)

    # keep order exactly like sample image
    bar_order = ['N86Y', 'Y184F', 'S1034C', 'N1042D', 'D1246Y']
    bar_order = [c for c in bar_order if c in mut_cols]

    x = range(len(bar_order))
    y = [mean_vals[c] for c in bar_order]
    yerr = [se_vals[c] for c in bar_order]
    cols = [bar_colors.get(c, 'white') for c in bar_order]

    axA.bar(x, y, yerr=yerr, capsize=6, width=0.55,
            color=cols, edgecolor='black', linewidth=2)

    axA.set_xticks(list(x))
    axA.set_xticklabels(bar_order, rotation=45, ha='right')
    axA.set_ylabel('prevalence', fontsize=16, fontweight='bold')
    axA.set_xlabel('mutations', fontsize=16, fontweight='bold')
    axA.set_ylim(0, 1.0)

    style_axes(axA)
    add_labels_on_bars(axA, dy=0.04)
    axA.text(0.01, 0.98, "A", transform=axA.transAxes, fontsize=14, fontweight="bold", va="top")

else:
    axA.text(0.5, 0.5, 'pfmdr1 mutation columns not found', ha='center', va='center')
    axA.set_title('')

# ======================
# Right panel: time series (N86Y squares black, Y184F triangles blue)
# ======================
plot_cols = [c for c in ['N86Y', 'Y184F'] if c in mut_cols]
if plot_cols and col_year in df_pfmdr_plot.columns:

    df_mdr_year = df_pfmdr_plot.groupby(col_year)[plot_cols].mean().sort_index()
    print(df_mdr_year)

    if 'N86Y' in df_mdr_year.columns:
        n86y = df_mdr_year['N86Y'].dropna()
        axB.plot(
            n86y.index, n86y.values,
            marker='s', markersize=6, color='black', linewidth=1, label='N86Y'
        )

    if 'Y184F' in df_mdr_year.columns:
        y184f = df_mdr_year['Y184F'].dropna()
        axB.plot(
            y184f.index, y184f.values,
            marker='^', markersize=7, color='blue', linewidth=1, label='Y184F'
        )


    axB.set_ylabel('prevalence of mdr-1 mutations', fontsize=16, fontweight='bold')
    axB.set_xlabel('year', fontsize=16, fontweight='bold')
    axB.set_ylim(0, 1.0)
    axB.set_xlim(2000)
    axB.legend(
        frameon=False,
        fontsize=12,
        loc='upper left'
    )


    style_axes(axB)

    # baseline at y=0 (thick)
    axB.axhline(0, color='black', linewidth=3)
    axB.text(0.01, 0.98, "B", transform=axB.transAxes, fontsize=14, fontweight="bold", va="top")

else:
    axB.text(0.5, 0.5, 'Year or mutation columns missing for time series', ha='center', va='center')

plt.tight_layout()
plt.show()
plt.savefig("plot/figure4_pfmdr1_prevalence.png", dpi=300)


## 7. Figure 5 – pfdhfr & pfdhps Mutations and Haplotypes

In [ ]:
YEAR_COL = "corrected year"

COL_TRIPLE = "51I & 59R & 108N"
COL_164L   = "I164L"

COL_A437G  = "A437G"
COL_K540E  = "K540E"
BAR_ORDER  = ["I431V", "S436A", "A437G", "K540E", "A581G", "A613S"]

MAGENTA = "#ff4fd8"
GREEN   = "lime"
BLUE    = "blue"
BLACK   = "black"
GRAY    = "gray"
WHITE   = "white"

def to_prop(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s.lower() in {"na", "n/a", "nan", "none", ""}:
        return np.nan
    s = s.replace("%", "").strip().replace(",", ".")
    try:
        v = float(s)
    except Exception:
        return np.nan
    # treat 33 as 33% if needed
    if v > 1.5:
        v /= 100.0
    return float(np.clip(v, 0, 1))

def to_year_mean(x):
    """
    '2016-2020' -> 2018.0
    '2012–2022' -> 2017.0  (en dash ok)
    '2021'      -> 2021.0
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s.lower() in {"na", "n/a", "nan", "none", ""}:
        return np.nan

    # normalize dash types
    s = s.replace("–", "-").replace("—", "-")
    if "-" in s:
        a, b = s.split("-", 1)
        try:
            return (float(a.strip()) + float(b.strip())) / 2.0
        except Exception:
            return np.nan
    try:
        return float(s)
    except Exception:
        return np.nan

def style_ax(ax, xlim=None, ylim=(0, 1.0), xlabel="year", ylabel=None, top_dotted=True):
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(2.5)
    ax.spines["bottom"].set_linewidth(2.5)
    ax.tick_params(axis="both", which="major", width=2.5, length=9, labelsize=10)
    if xlim is not None:
        ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_xlabel(xlabel, fontsize=11, fontweight="bold")
    if ylabel:
        ax.set_ylabel(ylabel, fontsize=11, fontweight="bold")
    if top_dotted:
        ax.axhline(ylim[1], linestyle=":", color="black", linewidth=1.8)

# --------------------
# LOAD
# --------------------
df_dhfr = pd.read_excel(S4_PATH, header=2, sheet_name="pfdhfr")
df_dhps = pd.read_excel(S4_PATH, header=2, sheet_name="pfdhps")
df_comb = pd.read_excel(S4_PATH, header=1, sheet_name="combined_haplotypes_dhfr_dhps")

# dhfr/dhps years are already numeric in your file (keep as int)
df_dhfr[YEAR_COL] = pd.to_numeric(df_dhfr[YEAR_COL], errors="coerce")
df_dhfr = df_dhfr.dropna(subset=[YEAR_COL]).copy()
df_dhfr[YEAR_COL] = df_dhfr[YEAR_COL].astype(int)

df_dhps[YEAR_COL] = pd.to_numeric(df_dhps[YEAR_COL], errors="coerce")
df_dhps = df_dhps.dropna(subset=[YEAR_COL]).copy()
df_dhps[YEAR_COL] = df_dhps[YEAR_COL].astype(int)

for c in df_dhfr.columns:
    if c != YEAR_COL:
        df_dhfr[c] = df_dhfr[c].apply(to_prop)

for c in df_dhps.columns:
    if c != YEAR_COL:
        df_dhps[c] = df_dhps[c].apply(to_prop)

# combined: IMPORTANT -> keep year as float because we may create means (2016-2020 -> 2018.0)
if "year of sample collection" in df_comb.columns and YEAR_COL not in df_comb.columns:
    df_comb[YEAR_COL] = df_comb["year of sample collection"].apply(to_year_mean)

if YEAR_COL in df_comb.columns:
    df_comb[YEAR_COL] = df_comb[YEAR_COL].apply(to_year_mean)
    df_comb = df_comb.dropna(subset=[YEAR_COL]).copy()

if "prevalence" in df_comb.columns:
    df_comb["prevalence"] = df_comb["prevalence"].apply(to_prop)

xmin = 2000
xmax = int(max(df_dhfr[YEAR_COL].max(), df_dhps[YEAR_COL].max()))
xmax = max(2022, xmax)

fig, axes = plt.subplots(2, 2, figsize=(12, 8), constrained_layout=True)
axA, axB = axes[0, 0], axes[0, 1]
axC, axD_outer = axes[1, 0], axes[1, 1]

# --------------------
# Panel A (pfdhfr)
# --------------------
if COL_TRIPLE in df_dhfr.columns:
    A = df_dhfr.groupby(YEAR_COL)[[c for c in [COL_TRIPLE, COL_164L] if c in df_dhfr.columns]].mean().sort_index()
    axA.plot(A.index, A[COL_TRIPLE], marker="o", linewidth=0.9, markersize=5.5,
             color=BLACK, label="triple mutation 51I, 59R, 108N")
    if COL_164L in A.columns:
        A164 = A[COL_164L].dropna()
        axA.plot(A164.index, A164.values, marker="s", linewidth=0.9, markersize=5.5,
                 color=BLUE, label="164L")
    style_ax(axA, xlim=(xmin, xmax), ylim=(0, 1.0), ylabel="prevalence of pfdhfr mutations")
    axA.axhline(0.1, linestyle=":", color=BLACK, linewidth=1.5)
    axA.legend(frameon=False, fontsize=9, loc="lower left")
axA.text(0.01, 0.98, "A", transform=axA.transAxes, fontsize=14, fontweight="bold", va="top")

# --------------------
# Panel B (pfdhps)
# --------------------
if COL_A437G in df_dhps.columns and COL_K540E in df_dhps.columns:
    B = df_dhps.groupby(YEAR_COL)[[COL_K540E, COL_A437G]].mean().sort_index()
    axB.plot(B.index, B[COL_K540E], marker="o", linewidth=0.9, markersize=5.5,
             color=BLUE, label="K540E")
    axB.plot(B.index, B[COL_A437G], marker="s", linewidth=0.9, markersize=5.5,
             color=BLACK, label="A437G")
    style_ax(axB, xlim=(xmin, xmax), ylim=(0, 1.0), ylabel="prevalence of pfdhps mutations")
    axB.axhline(0.2, linestyle=":", color=BLACK, linewidth=1.5)
    axB.legend(frameon=False, fontsize=10, loc="center right")
axB.text(0.01, 0.98, "B", transform=axB.transAxes, fontsize=14, fontweight="bold", va="top")

# --------------------
# Panel C (DHPS bars)
# --------------------
bar_cols = [c for c in BAR_ORDER if c in df_dhps.columns]
if bar_cols:
    means = df_dhps[bar_cols].mean(skipna=True)
    sems  = df_dhps[bar_cols].sem(skipna=True)
    color_map = {"I431V": BLACK, "S436A": BLUE, "A437G": MAGENTA, "K540E": WHITE, "A581G": GREEN, "A613S": GRAY}
    colors = [color_map.get(c, WHITE) for c in bar_cols]
    x = np.arange(len(bar_cols))
    axC.bar(x, means.values, yerr=sems.values, capsize=5, width=0.6,
            color=colors, edgecolor=BLACK, linewidth=2)
    axC.set_xticks(x)
    axC.set_xticklabels(bar_cols, rotation=45, ha="right")
    axC.set_ylim(0, 1.2)
    axC.set_ylabel("prevalence", fontsize=11, fontweight="bold")
    axC.set_xlabel("mutations", fontsize=11, fontweight="bold")
    axC.spines["top"].set_visible(False)
    axC.spines["right"].set_visible(False)
    axC.spines["left"].set_linewidth(2.5)
    axC.spines["bottom"].set_linewidth(2.5)
    axC.tick_params(axis="both", which="major", width=2.5, length=9, labelsize=10)
    for i, v in enumerate(means.values):
        if pd.notna(v):
            axC.text(i, v + 0.06, f"{v:.2f}", ha="center", va="bottom", fontsize=10)
axC.text(0.01, 0.98, "C", transform=axC.transAxes, fontsize=14, fontweight="bold", va="top")

# --------------------
# Panel D (FIXED)
# --------------------
axD_outer.set_visible(False)
gs = axD_outer.get_subplotspec().subgridspec(2, 1, height_ratios=[4, 1], hspace=0.05)
axD_top = fig.add_subplot(gs[0])
axD_bot = fig.add_subplot(gs[1], sharex=axD_top)

required = {"haplotype DHPS", "haplotype DHFR", "prevalence", YEAR_COL}
if required.issubset(df_comb.columns):
    d = df_comb.copy()

    dhps = d["haplotype DHPS"].astype(str)
    dhfr = d["haplotype DHFR"].astype(str)

    # flags DHPS (key mutations)
    d["dhps_has_437G"] = dhps.str.contains("437G", na=False)
    d["dhps_has_540E"] = dhps.str.contains("540E", na=False)
    d["dhps_has_581G"] = dhps.str.contains("581G", na=False)
    d["dhps_has_613S"] = dhps.str.contains("613S", na=False)

    # DHFR triple = contains all three tokens (NOT the literal string "51I, 59R, 108N")
    d["dhfr_is_triple"] = (
        dhfr.str.contains("51I", na=False)
        & dhfr.str.contains("59R", na=False)
        & dhfr.str.contains("108N", na=False)
    )

    partial_mask = (
        d["dhfr_is_triple"]
        & d["dhps_has_437G"]
        & ~d["dhps_has_540E"]
        & ~d["dhps_has_581G"]
        & ~d["dhps_has_613S"]
    )

    full_mask = (
        d["dhfr_is_triple"]
        & d["dhps_has_437G"]
        & d["dhps_has_540E"]
        & ~d["dhps_has_581G"]
        & ~d["dhps_has_613S"]
    )

    super_mask = (
        d["dhfr_is_triple"]
        & d["dhps_has_437G"]
        & d["dhps_has_540E"]
        & (d["dhps_has_581G"] | d["dhps_has_613S"])
    )

    # ---- group stats ----
    top_df = d.loc[partial_mask, [YEAR_COL, "prevalence"]].dropna()
    top_stats = (
        top_df.groupby(YEAR_COL)["prevalence"]
        .agg(mean="mean", sem=lambda s: s.sem())
        .sort_index()
    )

    bot1 = (
        d.loc[full_mask, [YEAR_COL, "prevalence"]]
        .dropna()
        .groupby(YEAR_COL)["prevalence"].mean()
        .sort_index()
    )

    bot2 = (
        d.loc[super_mask, [YEAR_COL, "prevalence"]]
        .dropna()
        .groupby(YEAR_COL)["prevalence"].mean()
        .sort_index()
    )

    # ---- TOP ----
    if len(top_stats):
        axD_top.errorbar(
            top_stats.index, top_stats["mean"].values,
            yerr=top_stats["sem"].values,
            fmt="o-", color=BLACK, ecolor=BLACK,
            elinewidth=1.2, capsize=4, capthick=1.2,
            linewidth=0.9, markersize=5.5,
            label="dhfr 51I+59R+108N,\ndhps 437G \n\n Partial resistance"
        )

    style_ax(axD_top, xlim=(2010, xmax), ylim=(0, 1.0), xlabel="year", top_dotted=True)
    axD_top.tick_params(labelbottom=False)

    # ---- BOTTOM ----
    if len(bot1):
        axD_bot.plot(
            bot1.index, bot1.values,
            linestyle="None", marker="s", markersize=7,
            color=BLUE,
            label="dhfr 51I+59R+108N,\ndhps 437G+540E \n\n Full resistance"
        )

    if len(bot2):
        axD_bot.plot(
            bot2.index, bot2.values,
            linestyle="None", marker="^", markersize=7,
            color=MAGENTA,
            label="dhfr 51I+59R+108N,\ndhps 437G+540E+581G\nor 613S\n\n Super resistance"
        )

    style_ax(axD_bot, xlim=(2010, xmax), ylim=(0, 0.06), xlabel="year", top_dotted=False)
    axD_bot.axhline(0.06, linestyle=":", color=BLACK, linewidth=1.8)
    axD_bot.set_yticks([0.00, 0.06])
    axD_top.set_xticks([2010, 2015, 2020,2025])

    # ---- legend ----
    h1, l1 = axD_top.get_legend_handles_labels()
    h2, l2 = axD_bot.get_legend_handles_labels()
    axD_bot.legend(
        h1 + h2, l1 + l2,
        frameon=False, fontsize=8.8,
        ncol=3, loc="upper center",
        bbox_to_anchor=(0.5, -0.75),
        handletextpad=0.6, columnspacing=1.2
    )

axD_top.text(0.01, 0.98, "D", transform=axD_top.transAxes,
             fontsize=14, fontweight="bold", va="top")

plt.show()
plt.savefig("plot/figure_5_dhfr_dhps_prevalence.png", dpi=300)


## 8. Figure S1 – Additional pfdhps Mutations (I431V, S436A, A581G, A613S)

In [ ]:
# --- Figure S1: Additional dhps mutations over time (4 panels) ---

extra_cols = [c for c in ["I431V", "S436A", "A581G", "A613S"] if c in df_dhps.columns]

if YEAR_COL in df_dhps.columns and extra_cols:
    df_s1 = df_dhps[[YEAR_COL] + extra_cols].copy()

    for c in extra_cols:
        df_s1[c] = df_s1[c].apply(to_prop)

    s1_year = df_s1.groupby(YEAR_COL)[extra_cols].mean().sort_index()

    fig_s1, axes = plt.subplots(2, 2, figsize=(10, 7), constrained_layout=True)
    axes = axes.ravel()

    marker_map = {
        "I431V": "^",
        "S436A": "s",
        "A581G": "^",
        "A613S": "^"
    }

    panel_labels = ["A", "B", "C", "D"]

    for i, (ax, col) in enumerate(zip(axes, extra_cols)):
        ax.plot(
            s1_year.index, s1_year[col].values,
            marker=marker_map.get(col, "^"),
            linewidth=0.9, markersize=5.5,
            color="black",
            label=col
        )

        style_ax(
            ax,
            xlim=(2000, xmax),
            ylim=(0, 1.0),
            xlabel="year",
            ylabel="prevalence",
            top_dotted=True
        )

        ax.set_xticks([2000, 2005, 2010, 2015, 2020, 2025])

        # panel label (A–D)
        ax.text(
            0.02, 0.96, panel_labels[i],
            transform=ax.transAxes,
            fontsize=13, fontweight="bold",
            va="top"
        )

        # legend inside each panel
        ax.legend(
            frameon=False,
            fontsize=9,
            loc="upper right"
        )

    # hide unused panels if < 4 columns
    for j in range(len(extra_cols), 4):
        axes[j].set_visible(False)

    plt.show()
    plt.savefig("plot/figure_S1_additional_dhps_mutations_over_time.png", dpi=300)
else:
    print(f"Missing {YEAR_COL} or required columns. Found extra_cols={extra_cols}")


In [ ]:
YEAR_COL = "corrected year"   
df_k13 = pd.read_excel(S5_PATH)  

df_k13[YEAR_COL] = df_k13["corrected year"].apply(parse_year_mean)

df_k13["prevalence"] = df_k13["prevalence"].apply(prop_to_float)

N_COL = "sample size Nigeria only"
df_k13[N_COL] = pd.to_numeric(df_k13[N_COL], errors="coerce")

df_k13["mut_label"] = (
    df_k13["WT AA"].astype(str).str.strip() +
    df_k13["codon"].astype(str).str.strip() +
    df_k13["mutant AA"].astype(str).str.strip()
)

if "mutations yes/no" in df_k13.columns:
    df_k13 = df_k13[df_k13["mutations yes/no"].astype(str).str.lower().eq("yes")].copy()


In [ ]:

def style_ax_simple(ax, xlim=None, ylim=(0,1.0), xlabel="year", ylabel="prevalence"):
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(2.5)
    ax.spines["bottom"].set_linewidth(2.5)
    ax.tick_params(axis="both", which="major", width=2.5, length=9, labelsize=10)
    if xlim: ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_xlabel(xlabel, fontsize=11, fontweight="bold")
    ax.set_ylabel(ylabel, fontsize=11, fontweight="bold")
    ax.axhline(ylim[1], linestyle=":", color="black", linewidth=1.8)

tmp = df_k13.dropna(subset=[YEAR_COL, "prevalence", N_COL]).copy()
tmp["mut_count"] = np.rint(tmp["prevalence"] * tmp[N_COL]).astype(int)
top4 = (tmp.groupby("mut_label")["mut_count"].sum().sort_values(ascending=False).head(4).index.tolist())


def weighted_prev(g):
    n = g[N_COL].sum()
    if n <= 0: return np.nan
    return (g["prevalence"] * g[N_COL]).sum() / n

k13_year = (
    tmp[tmp["mut_label"].isin(top4)]
    .groupby([YEAR_COL, "mut_label"])
    .apply(weighted_prev)
    .rename("prev_w")
    .reset_index()
)

years = sorted(k13_year[YEAR_COL].dropna().unique())
xmin, xmax = int(min(years)), int(max(years))

fig, axes = plt.subplots(2, 2, figsize=(10, 7), constrained_layout=True)
axes = axes.ravel()
panel_letters = ["A", "B", "C", "D"]
marker_cycle = ["o", "s", "^", "D"]

for i, mut in enumerate(top4):
    ax = axes[i]
    d = k13_year[k13_year["mut_label"] == mut].sort_values(YEAR_COL)
    ax.plot(d[YEAR_COL], d["prev_w"], color="black", marker=marker_cycle[i], linewidth=0.9, markersize=5.5, label=mut)

    style_ax_simple(ax, xlim=(xmin, xmax), ylim=(0,1.0))
    ax.legend(frameon=False, fontsize=10, loc="upper left")
    ax.text(0.01, 0.98, panel_letters[i], transform=ax.transAxes,
            fontsize=14, fontweight="bold", va="top")


for j in range(len(top4), 4):
    axes[j].set_visible(False)

plt.show()
plt.savefig("plot/figure_S2_pfkelch13_top4_mutations_over_time.png", dpi=300)